# INTRODUCTION

### Data
The data is about Asteroids - NeoWs.
NeoWs (Near Earth Object Web Service) is a RESTful web service for near earth Asteroid information. With NeoWs a user can: search for Asteroids based on their closest approach date to Earth, lookup a specific Asteroid with its NASA JPL small body id, as well as browse the overall data-set.

Acknowledgements
Data-set: All the data is from the (http://neo.jpl.nasa.gov/). This API is maintained by SpaceRocks Team: David Greenfield, Arezu Sarvestani, Jason English and Peter Baunach.
 
### Tasks
Based on the information within the dataset we want to performs two tasks:
- Develop a **model that predicts if an asteroid is going to be hazardous or not**
- Identify which **features are more relevant towards the classfication** on point 1

To tackle this two tasks I'll use the methods listed below, and extract the results from the method with best performance on unseen data
- Logistic Regression
- Decision Tree
- Random Forest
- SVM
- XGBossting
 

In [ ]:
!pip install xgboost


# Libraries & Data Import

In [ ]:
# Programming
import pandas as pd
import numpy as np

# Machine Learning | sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Other
import missingno as msno
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('nasa.csv',
                 parse_dates=['Close Approach Date', 'Orbit Determination Date', 'Epoch Date Close Approach'])

In [ ]:
# Fixing seed for reproducibility
seed = 1234

# PRE-PROCESSING

# Step 1: Data Inspection

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.info()

## Correlation among the variables

In [ ]:
sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr())
plt.show()

In [ ]:
df.describe()

# Step 2: Check Missing Values

In [ ]:
# Check for missing values
print(df.isnull().sum())

# Step 3: Imputing

This step is not necessary for this specific dataset, as there are no missing values.

# Step 4: Dimensionality Reduction

In [ ]:
# Dropping completely correlated features and datetime features
df = df.drop(['Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)', 'Est Dia in Miles(max)', 'Est Dia in Feet(min)', 'Est Dia in Feet(max)', 'Est Dia in KM(max)',
              'Relative Velocity km per sec', 'Miles per hour',
              'Miss Dist.(Astronomical)', 'Miss Dist.(lunar)', 'Miss Dist.(miles)',
              'Semi Major Axis',
              'Neo Reference ID', 'Name',
              'Close Approach Date', 'Epoch Date Close Approach', 'Orbit ID', 'Orbit Determination Date', 'Epoch Osculation','Perihelion Time'],axis=1)

# Plotting feature correlation with reduced dataset
sns.set(rc={'figure.figsize':(30,20)})
sns.heatmap(df.corr())
plt.show()
plt.close()

In [ ]:
# To get a more clear picture of the correlation with Hazardous Nature
df.corr()['Hazardous'][:-1].sort_values().plot(kind='bar')
plt.show()

# Exploratory Data Analysis

Hazardous Nature  
Is the asteroid hazardous? (True or False)

In [ ]:
# get the values
print(df['Hazardous'].value_counts())
print('\n')

# plot
lab = 'False','True'
plt.pie(df['Hazardous'].value_counts(), labels = lab, colors = ['green','red'],
        autopct='%1.1f%%')
plt.title('Hazardous Nature')
plt.show()

The datset contains 16.1% of hazardous asteroids and 83.9% of non-hazardous asteroids.

Orbit Uncertainity  
A measure of the uncertainty ('measurement errors') in the calculated orbit

In [ ]:
import plotly.express as px
# get the values
print(df['Orbit Uncertainity'].value_counts())
print('\n')

# plot
df1 = df.groupby(["Orbit Uncertainity"]).count().reset_index()

fig = px.bar(df1,
             y=df.groupby(["Orbit Uncertainity"]).size()/len(df)*100,
             x="Orbit Uncertainity")
fig.update_layout(yaxis_title='percentage',
                  title_text='Orbit Uncertainity')
fig.show()

Orbit Uncertainity vs. Hazardous Nature

In [ ]:
# plot
df.groupby('Orbit Uncertainity')['Hazardous'].value_counts(normalize=True).unstack('Hazardous').plot.bar(stacked=True, title='Orbit Uncertainity vs. Hazardous Nature')
plt.show()

Absolute Magnitude  
Measure of the luminosity of an asteroid, on an inverse logarithmic astronomical magnitude scale.

In [ ]:
# plot
sns.histplot(df['Absolute Magnitude']).set_title('Absolute Magnitude')
plt.show()

Absolute Magnitude vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Absolute Magnitude",data=df,palette=('green','red')).set_title('Absolute Magnitude vs. Hazardous Nature')
plt.show()

Minimum Orbit Intersection  
The closest distance between Earth and the asteroid in their respective orbits (in astronomical units)

In [ ]:
# plot
sns.histplot(df['Minimum Orbit Intersection']).set_title('Minimum Orbit Intersection')
plt.show()

Minimum Orbit Intersection vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Minimum Orbit Intersection",data=df,palette=('green','red')).set_title('Minimum Orbit Intersection vs. Hazardous Nature')
plt.show()

Perihelion Distance  
Distance of point in asteroid's orbit which is closest to the Sun

In [ ]:
# plot
sns.histplot(df['Perihelion Distance']).set_title('Perihelion Distance')
plt.show()

Perihelion Distance vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Perihelion Distance",data=df,palette=('green','red')).set_title('Perihelion Distance vs. Hazardous Nature')
plt.show()

Est Dia in KM(min)  
Minimum estimated diameter of the asteroid

In [ ]:
# plot
sns.histplot(df['Est Dia in KM(min)']).set_title('Estimated diameter(min)')
plt.show()

Est Dia in KM(min) vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Est Dia in KM(min)",data=df,palette=('green','red')).set_title('Estimated diameter(min) vs. Hazardous Nature')
plt.show()

Eccentricity  
A value which specifies by how much the asteroid's orbit deviates from a perfect circle

In [ ]:
# plot
sns.histplot(df['Eccentricity']).set_title('Eccentricity')
plt.show()

Eccentricity vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Eccentricity",data=df,palette=('green','red')).set_title('Eccentricity vs. Hazardous Nature')
plt.show()

Relative Velocity  
Asteroid's velocity relative to earth

In [ ]:
# plot
sns.histplot(df['Relative Velocity km per hr']).set_title('Relative Velocity')
plt.show()

Relative Velocity vs. Hazardous Nature

In [ ]:
# plot
sns.boxplot(x="Hazardous", y="Relative Velocity km per hr",data=df,palette=('green','red')).set_title('Relative Velocity vs. Hazardous Nature')
plt.show()

# Step 5: Categorical Feature Encoding

In [ ]:
# Encoding the target variable
l_enc = LabelEncoder()
df['Hazardous'] = l_enc.fit_transform(df.Hazardous) 
print('Hazardous == True -> 1')
print('Hazardous == False -> 0\n')

# Checking if the other categorical features need to be encoded
print(df['Orbiting Body'].unique())
print(df['Equinox'].unique())
print('\n')
# Removing them since there is only a single value that is identical across all observations
df = df.drop(['Orbiting Body', 'Equinox'], axis=1)

# Check after all the changes
print(df.info())
df.head()

# Step 6: Train/Test Split

In [ ]:
# Creating the Features/Label split as numpy arrays
features = df.drop('Hazardous', axis=1).values
label = df.Hazardous.values


# Split the data into training, validation, and test sets
training_features, val_features, training_label, val_label = train_test_split(features, label,
                                                                                      test_size=0.1,
                                                                                      stratify=label,
                                                                                      random_state=seed)
training_features, test_features, training_label, test_label = train_test_split(training_features, training_label,
                                                                          test_size=0.2,
                                                                          stratify=training_label,
                                                                          random_state=seed)


# Getting feature labels for future plotting
df_graph = df.copy()
feature_names = df_graph.drop('Hazardous', axis=1).columns.tolist()
del df_graph

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE

# Perform feature selection using SelectKBest and f_classif
selector = SelectKBest(score_func=f_classif, k=5)  # Select the top 10 features
selected_features = selector.fit_transform(training_features, training_label)
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = [feature_names[i] for i in selected_feature_indices]

# Update training and test features with the selected features
training_features = training_features[:, selected_feature_indices]
test_features = test_features[:, selected_feature_indices]
val_features = val_features[:, selected_feature_indices]

# Print the selected feature names
print('Selected Features:', selected_feature_names)


In [ ]:
# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=seed)
training_features, training_label = smote.fit_resample(training_features, training_label)

# Print the class distribution after oversampling
print('Class Distribution after SMOTE:')
print(pd.Series(training_label).value_counts())


# MODELS

# Logistic Regression

In [ ]:
# Creating the pipeline
logreg_pipe = Pipeline([('Scaling', StandardScaler()),
                        ('LogReg', LogisticRegression())])

# Creating hyperparameter options
logreg_params = {'LogReg__C': np.arange(0, 10, 0.1),
                 'LogReg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                 'LogReg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
                 'LogReg__random_state': [seed]}

# GrideSearcCV
logreg_grid = GridSearchCV(estimator=logreg_pipe, param_grid=logreg_params,
                           scoring='accuracy', cv=5)
logreg_grid.fit(training_features, training_label)
logreg_opt_param = logreg_grid.best_params_
logreg_best_score = (logreg_grid.best_score_*100).round(2)
logreg_best_est = logreg_grid.best_estimator_

# Score on holdout data
logreg_holdout_score = (logreg_grid.score(test_features, test_label)*100).round(2)

print('Optimal Hyperparameters:')
print(logreg_opt_param)
print('Optimal Estimator:')
print(logreg_best_est)
print('\n')
print('Training Accuracy {}'.format(logreg_best_score))
print('Testing Accuracy {}'.format(logreg_holdout_score))

# Decision Tree

In [ ]:
# Creating hyperparameter options
dectree_params = {'max_depth': np.arange(0, 20, 1),
                  'criterion': ['gini', 'entropy'],
                  'min_samples_leaf': np.arange(0, 1, 0.05),
                  'random_state': [seed]}

# GrideSearcCV
dectree_grid = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=dectree_params,
                            scoring='accuracy', cv=5)
dectree_grid.fit(training_features, training_label)
dectree_opt_param = dectree_grid.best_params_
dectree_best_score = (dectree_grid.best_score_*100).round(2)
dectree_best_est = dectree_grid.best_estimator_
dectree_feat_imp = dectree_best_est.feature_importances_

# Score on holdout data
dectree_holdout_score = (dectree_grid.score(test_features, test_label)*100).round(2)

print('Optimal Hyperparameters:')
print(dectree_opt_param)
print('Optimal Estimator:')
print(dectree_best_est)
print('\n')
print('Training Accuracy {}'.format(dectree_best_score))
print('Testing Accuracy {}'.format(dectree_holdout_score))

# Random Forest

In [ ]:
# Creating hyperparameter options
rf_params = {'max_depth': np.arange(0, 20, 1),
             'criterion': ['gini', 'entropy'],
             'min_samples_leaf': np.arange(0, 1, 0.05),
             'random_state': [seed],
             'n_estimators': np.arange(0, 10, 1)}

# GrideSearcCV
rf_grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params,
                       scoring='accuracy', cv=5)
rf_grid.fit(training_features, training_label)
rf_opt_param = rf_grid.best_params_
rf_best_score = (rf_grid.best_score_*100).round(2)
rf_best_est = rf_grid.best_estimator_
rf_feat_imp = rf_best_est.feature_importances_

# Score on holdout data
rf_holdout_score = (rf_grid.score(test_features, test_label)*100).round(2)

print('Optimal Hyperparameters:')
print(rf_opt_param)
print('Optimal Estimator:')
print(rf_best_est)
print('\n')
print('Training Accuracy {}'.format(rf_best_score))
print('Testing Accuracy {}'.format(rf_holdout_score))

In [ ]:
# Plotting feature importnace
plt.barh(selected_feature_names, rf_feat_imp)
plt.show()
plt.close

# XGBoosting

In [ ]:
# Creating hyperparameter options
xgb_params = {'max_depth': np.arange(0, 5, 1),
              'objective': ['binary:logistic'],
              'random_state': [seed],
              'alpha': [0, 0.01, 0.1, 1],
              'lambda': [0, 0.01, 0.1, 1],
              'subsample': [0.25, 0.5, 0.75],
              'colsample_bytree': [0.25, 0.5, 0.75],
              'eval_metric': ['logloss']}

# GrideSearcCV
xgb_grid = GridSearchCV(estimator=XGBClassifier(), param_grid=xgb_params,
                            scoring='accuracy', cv=5)
xgb_grid.fit(training_features, training_label)
xgb_opt_param = xgb_grid.best_params_
xgb_best_score = (xgb_grid.best_score_*100).round(2)
xgb_best_est = xgb_grid.best_estimator_
xgb_feat_imp = xgb_best_est.feature_importances_

# Score on holdout data
xgb_holdout_score = (xgb_grid.score(test_features, test_label)*100).round(2)

print('Optimal Hyperparameters:')
print(xgb_opt_param)
print('Optimal Estimator:')
print(xgb_best_est)
print('\n')
print('Training Accuracy {}'.format(xgb_best_score))
print('Testing Accuracy {}'.format(xgb_holdout_score))

In [ ]:
# Plotting feature importance
plt.barh(selected_feature_names, xgb_feat_imp)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


from sklearn.ensemble import VotingClassifier

# Hybrid Model
hybrid_models = [('DecisionTree', dectree_best_est), ('RandomForest', rf_best_est), ('XGBoost', xgb_best_est)]
hybrid_model = VotingClassifier(hybrid_models)
hybrid_model.fit(training_features, training_label)
hybrid_predictions = hybrid_model.predict(test_features)
hybrid_accuracy = accuracy_score(test_label, hybrid_predictions)

print('Hybrid Model:')
print('Accuracy:', hybrid_accuracy)
print('Classification Report:')
print(classification_report(test_label, hybrid_predictions))
print('Confusion Matrix:')
print(confusion_matrix(test_label, hybrid_predictions))

# Cross Validation for Hybrid Model
hybrid_cv_scores = cross_val_score(hybrid_model, features, label, cv=5, scoring='accuracy')
mean_hybrid_cv_score = np.mean(hybrid_cv_scores)

print('Cross Validation Accuracy for Hybrid Model:', mean_hybrid_cv_score)


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

# Define the SGD classifier with the desired optimizer
sgd_classifier = SGDClassifier(loss='log', max_iter=1000, random_state=42)


# Update hybrid models with optimizers
hybrid_models_optimizers = [('SGD', sgd_classifier)]

# Lists to store accuracy values
train_acc_values = []
val_acc_values = []

# Iterate over the optimizers
for optimizer, model in hybrid_models_optimizers:
    hybrid_model.estimators = hybrid_models + [(optimizer, model)]
    
    # Fit the hybrid model
    hybrid_model.fit(training_features, training_label)
    
    # Predict on training and validation data
    train_predictions = hybrid_model.predict(training_features)
    val_predictions = hybrid_model.predict(test_features)
    
    # Calculate accuracy for training and validation data
    train_accuracy = accuracy_score(training_label, train_predictions)
    val_accuracy = accuracy_score(test_label, val_predictions)
    
    # Store the accuracy values
    train_acc_values.append(train_accuracy)
    val_acc_values.append(val_accuracy)
    
    # Print the results
    print('Optimizer:', optimizer)
    print('Training Accuracy:', train_accuracy)
    print('Validation Accuracy:', val_accuracy)
    print('-----------------------')


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import numpy as np

# Define the SGD classifier with the desired optimizer
sgd_classifier = SGDClassifier(loss='log', max_iter=1000, random_state=42)

# Update hybrid models with optimizers
hybrid_models_optimizers = [('SGD', sgd_classifier)]

# Lists to store accuracy values
train_acc_values = []
val_acc_values = []

# Set the number of epochs
num_epochs = 10

# Iterate over the optimizers
for optimizer, model in hybrid_models_optimizers:
    # Create a new instance of VotingClassifier for each optimizer
    hybrid_model = VotingClassifier(estimators=hybrid_models + [(optimizer, model)])
    
    # Lists to store accuracy values for each epoch
    train_acc_epochs = []
    val_acc_epochs = []
    
    # Iterate over epochs
    for epoch in range(1, num_epochs + 1):
        # Fit the hybrid model
        hybrid_model.fit(training_features, training_label)
        
        # Predict on training and validation data
        train_predictions = hybrid_model.predict(training_features)
        val_predictions = hybrid_model.predict(test_features)
        
        # Calculate accuracy for training and validation data
        train_accuracy = accuracy_score(training_label, train_predictions)
        val_accuracy = accuracy_score(test_label, val_predictions)
        
        # Append accuracy values for the current epoch
        train_acc_epochs.append(train_accuracy)
        val_acc_epochs.append(val_accuracy)
        
    # Print the results
    print('Optimizer:', optimizer)
    print('Training Accuracy:', train_accuracy)
    print('Validation Accuracy:', val_accuracy)
    print('-----------------------')
    
    # Append accuracy values for all epochs
    train_acc_values.append(train_acc_epochs)
    val_acc_values.append(val_acc_epochs)

# Convert accuracy lists to numpy arrays
train_acc_values = np.array(train_acc_values)
val_acc_values = np.array(val_acc_values)

# Plot the accuracy evaluation graph
epochs = np.arange(1, num_epochs + 1)
plt.plot(epochs, train_acc_values.T, label='Train Accuracy')
plt.plot(epochs, val_acc_values.T, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Evaluate the model on the validation set
val_predictions = hybrid_model.predict(val_features)
val_accuracy = accuracy_score(val_label, val_predictions)
print('Validation Accuracy:', val_accuracy)

# CONCLUSIONS

### Model Performance on Unseen Data
 Model | Accuracy (%)
 - | -
Logistic Regression | 95.47
Decision Tree | 99.44
SVM | 94.99
Random Forest | 99.47
XGBoosting | **99.49**

As we can see above, **XGBoosting** has provided the best performance on unseen data, and thus is the best model for this classifcation problem out of those tested. Nonetheless the difference amongst the three tree based models (Decision Tree / Random Forest / XGBoosting) is almost negligible. 

In an scenario were the model was going to be deployed into production, it would be interesting to look into the relation between the different accuracy performances and resource consumption. Given the small performance difference, an argument can be made that the best model, out of the three tree based models, is the one that consumes less resources.

### Feature Importance
Looking into each tree based model we can see a pattern on feature importance of the best estimator found for each model.

Found on 100% of the best models:
- Minimum Orbit Intersection
- Absolute Magnitude

Found on 33% of the best models:
- Orbit ID
- Perihelion Distance
- Est dia in KM(min). This feature is the one I selected to capture size. Given the perfect correlation between size features (same measurement, different units) any of them would yield the same result, as long as only one is selected as part of the training dataset.


### Overfitting
- All models present some degree of overfitting (train accuracy > test accuracy). Nonetheless the differences are relatively small, less than 1 percentage point (except for SVM). Given this I would not consider that the selected model presents a significant overfitting issue.


In [ ]:
import joblib
joblib.dump(hybrid_model,'model.pkl')